# Latent Dirichlet Allocation

Los modelos temáticos agrupan las palabras de un documento en diferentes temas. Por ejemplo, para la frase 

> A este chico le encanta jugar en el parque y le gusta el helado.

se puede decir que consta de dos temas importantes: 1) La comida, debido al "helado", y 2) Las actividades, debido al "juego" y al "parque". 

**Latent Dirichlet Allocation (LDA)** es un algoritmo de aprendizaje no supervisado utilizado para descubrir diferentes temas y sus indicadores asociados (palabras relacionadas con el tema) en una colección de documentos. **LDA** se basa en la idea de que las palabras a menudo tienen fuertes relaciones semánticas con ciertos temas, por lo que los temas en un documento dado consistirán en un grupo de palabras similares.

Al igual que en el algoritmo *K-means*, LDA requiere que elijamos el número de temas para que los descubra, y agrupa las palabras en el corpus de texto (un conjunto de documentos) que frecuentemente ocurren juntos dentro del tema. **LDA** asume que un documento es una mezcla de un conjunto de temas latentes (desconocidos), y cada tema es otra mezcla de palabras (colección de palabras que ayudan a identificar el tema).

## El proceso de extracción de temas

LDA es conocido por ser un modelo generativo, en el contexto del análisis de texto esto asume que los documentos son generados a través de algún proceso estadístico. 

Dado un documento $d$ en un corpus $D$ (un conjunto de documentos), entonces $d$ está generado por,

1. Número de palabras en el documento $d$, representado por $N_{d}$, que sigue una distribución de Poisson $N_{d} \sim Poisson(\eta).$

2. La mezcla de temas en el documento $d$, representado por $\theta_{d},$ sigue una distribución $\theta_{d} \sim Dirichlet(\alpha)$

3. Se asigna a cada palabra $w_{i}, i = 1, \ldots, N_{d},$ un tema, $z_{i},$ de forma que sea consistente con la distribución tema-documento definida en 2. Es decir, $z_{i} \sim Multinomial(\theta_{d})$

4. Una vez que se conoce el tema $z_{i}$ de cada palabra $w_{i},$ podemos extraer palabras $w_{i}$ de la distribución tema-palabra $\phi(\beta)$. Es decir, se elige $w_{i}$ con probabilidad $Pr(w_{i}|z_{i},\beta)$

Para resumir, LDA asume que un documento es una mezcla de temas, donde los temas se extraen de la distribución de temas-documento, y los temas consisten en palabras, donde las palabras se extraen de la distribución de temas-palabras. En la práctica ya tenemos un corpus de texto, un conjunto de documentos. Por lo tanto, normalmente no estamos interesados en generar nuevos documentos, sino más bien en hacer inferencias sobre cómo se genera el documento a partir de diversos temas y palabras.

In [ ]:
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt

x = np.arange(0, 100, 2)    
y = poisson.pmf(x, mu=42.7)

plt.plot(x, y, 'bo', ms=8)
plt.vlines(x, 0, y, colors='b', lw=1, alpha=0.5)

## El algoritmo

En LDA se supone que la distribución *a priori* sobre los temas de un corpus de texto sigue una distribución de Dirichlet. Esta distribución *a priori* proporciona la idea de que los documentos incluyen una mezcla de temas, y cada tema tiene una distribución sobre palabras que son similares. Hemos asumido un proceso estadístico en el que se generan documentos, por lo que ahora podemos combinar esta suposición con datos reales para encontrar la composición de los temas que aparecen en cada uno de nuestros documentos.

El parámetro clave que debemos encontrar es la probabilidad de asignar una palabra de un documento dado a un tema en particular, dadas nuestras suposiciones previas y el vocabulario de nuestro *corpus* (palabras únicas en todos los documentos).

De acuerdo a todo esto, el algoritmo para la detección de temas en documentos quedaría de la siguiente forma:

1. Se asigna de manera aleatoria uno de los $K$ temas a cada una de las palabras de los documentos. Esta es la asignación inicial palabra-tema, que se irá actualizando durante la ejecución del algoritmo.
2. Para cada documento $d$:
3. Para cada palabra $w$ del documento $d$ se le asigna un tema de acuerdo a la asignación inicial tema-palabra.
4. Ahora, se reasigna cada palabra $w$ del documento $d$ al tema $k$ con probabilidad
$$Pr(z_{(d,n)} = k | z_{-(d,n)},W;\alpha,\beta) \propto (n_{d,(.)}^{k,-(d,n)} + \alpha)\frac{n_{(.),w}^{k,-(d,n)} + \beta}{\sum_{w=1}^{V} n_{(.),w}^{k,-(d,n)} + \beta V},$$

donde $z_{(d,n)}$ es el topic asignado a la palabra *n-esima* del documento $d$, $z_{-(d,n)}$ son los temas asignados al resto de palabras del documento $d$, $V$ es el conjunto de todas las palabras encontradas en el corpus, $n_{d,(.)}^{k,-(d,n)}$ es el número de palabras del documento $d$ asignadas al tema $k$, excluyendo $z_{(d,n)}$ y, de manera análoga, $n_{(.),w}^{k,-(d,n)}$ es el número de documentos en los cuales la palabra $w$ se asigna al tema $k$, excluyendo $z_{(d,n)}.$
5. Actualizar la asignación inicial tema-palabra con la nueva asignación obtenida del paso 4.
6. Repetir los pasos 2-5 el número de iteraciones deseado.

En cada iteración nuestro modelo generativo se hará más preciso y comenzará a captar tendencias en nuestros datos. Una vez que el algoritmo termina, tendremos la distribución de temas-documentos, y también la distribución de temas-trabajo.


## Aplicando Latent Dirichlet Allocation para reducir la dimensionalidad de un Bag of Words


En primer lugar, como es usual, importamos las librerías que se van a utilizar, algunos parámetros y funciones auxiliares:


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 20000
n_features = 10000
n_components = 20
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()



Cargamos el conjunto de datos de los 20 grupos de noticias y lo vectorizamos. Utilizamos algunas heurísticas para filtrar los términos inútiles desde el principio: los mensajes se despojan de encabezados, pies de página y respuestas citadas, y se eliminan palabras comunes en inglés, palabras que aparecen en un solo documento o en al menos el 95% de los documentos.

In [ ]:
data = fetch_20newsgroups(shuffle=True, random_state=1337,
                             remove=('headers', 'footers', 'quotes'))
data_samples = data.data[:n_samples]

Vamos a usar la frecuencia de las palabras con `CountVectorizer`:

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(data_samples)

Y por último usamos las frecuencias para lanzar **LDA** y obtener el conjunto de temas que encontramos en los documentos:

In [ ]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=1337)
lda.fit(tf)

Veamos cuáles son los temas identificados por el algoritmo:

In [ ]:
print("Temas identificados:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

---

Creado por **Raúl Lara** (raul.lara@upm.es)

<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png">